In [21]:
import sys, os
import pandas as pd
import plotly.graph_objects as go

repo_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

from utils import (
    ray_to_percent,
    fetch_interest_rate_history,
    get_price
)

CHAIN = 'hyperEvm'
TOKEN = '0xb88339CB7199b77E23DB6E890353E22632Ba630f'
BASE_URL = 'https://api.hyperlend.finance'

In [22]:
raw_data, TOKEN_CS = fetch_interest_rate_history(CHAIN, TOKEN, BASE_URL)
df_interest = pd.DataFrame([
    {
        'timestamp': pd.to_datetime(entry['timestamp'], unit='ms'),
        'liquidity_rate_%': ray_to_percent(pool['currentLiquidityRate']),
        'borrow_rate_%': ray_to_percent(pool['currentVariableBorrowRate'])
    }
    for entry in raw_data if (pool := entry.get(TOKEN_CS))
])

df_interest = df_interest.sort_values('timestamp').reset_index(drop=True)
print(f'Fetched {len(df_interest)} hourly samples for token {TOKEN_CS} (first={df_interest['timestamp'].iloc[0] if not df_interest.empty else None})')


Fetched 1037 hourly samples for token 0xb88339CB7199b77E23DB6E890353E22632Ba630f (first=2025-09-17 19:39:07.480000)


In [23]:
current_price = get_price("HYPE")

loan_date = '2025-10-16 00:00:00' # takes all your loans from this date onwards

df = pd.read_csv('trade_history.csv')
df["time"] = pd.to_datetime(df["time"], format="%d/%m/%Y - %H:%M:%S")  # datetime
df["coin"] = df["coin"].astype("category")                             # category (string with few unique values)
df["dir"] = df["dir"].astype("category")                               # category
df[["px", "sz", "ntl", "fee", "closedPnl"]] = df[["px", "sz", "ntl", "fee", "closedPnl"]].astype(float)  # numeric

df['time'] = pd.to_datetime(df['time'])

hype_pairs = ['HYPE/USDC', 'HYPE/USDH', 'HYPE/USDT']
mask = (df['coin'].isin(hype_pairs)) & (df['time'] > loan_date) 

subset = df.loc[mask].copy()

buys = subset[subset['dir'].str.lower() == 'buy']
sells = subset[subset['dir'].str.lower() == 'sell']

total_bought = buys['sz'].sum()
total_sold = sells['sz'].sum()

notional_bought = (buys['px'] * buys['sz']).sum()
avg_buy_price = notional_bought / total_bought if total_bought > 0 else 0

total_fee = abs(subset['fee'].sum())

net_loan_hype = total_bought - total_sold

loan_pnl = (current_price - avg_buy_price) * net_loan_hype - total_fee

print(f"Total HYPE bought: {total_bought:.2f}")
print(f"Total HYPE sold: {total_sold:.2f}")
print(f"Net HYPE held: {net_loan_hype:.2f}")
print(f"Average buy price: {avg_buy_price:.2f}")
print(f"Current price: {current_price:.2f}")
print(f"Unrealized PnL: ${loan_pnl:.2f}")
print(f"Total fees: {total_fee:.2f}")


Total HYPE bought: 1788.77
Total HYPE sold: 0.00
Net HYPE held: 1788.77
Average buy price: 39.74
Current price: 41.77
Unrealized PnL: $3631.20
Total fees: 0.81


In [24]:
loan_amt = current_price * net_loan_hype

interest_mask = df_interest['timestamp'] > pd.to_datetime(loan_date)
interest_subset = df_interest.loc[interest_mask].copy()

interest_subset['borrow_notional'] = loan_amt * interest_subset['borrow_rate_%'] / (365*24) / 100

daily_interest = (
    interest_subset
    .groupby(interest_subset['timestamp'].dt.date)['borrow_notional']
    .sum()
    .reset_index()
    .rename(columns={'timestamp': 'date', 'borrow_notional': 'daily_interest'})
)

period_days = (daily_interest['date'].max() - daily_interest['date'].min()).days + 1

total_interest = daily_interest['daily_interest'].sum()

annualised_interest = total_interest * (365 / period_days)

print(f"Total interest accrued over period: ${total_interest:.2f}")
print(f"Annualised interest: ${annualised_interest:.2f}")
if annualised_interest > 1000:
    print('fuck')
else:
    print('you good bro')


Total interest accrued over period: $319.64
Annualised interest: $5555.68
fuck


In [19]:
trace = go.Bar(
    x=daily_interest['date'],
    y=daily_interest['daily_interest'],
    text=daily_interest['daily_interest'].round(2),
    textposition='outside',
    name='Daily Interest',
    marker_color='steelblue'
)

layout = go.Layout(
    title=f'Daily Borrow Interest on ${loan_amt:,} HYPE/USDC Loan',
    xaxis=dict(title='Date', tickangle=-45),
    yaxis=dict(title='Daily Interest ($)'),
    margin=dict(l=60, r=30, t=60, b=60)
)

fig = go.Figure(data=[trace], layout=layout)

fig.show()


Including previous purchases

In [20]:
date = '2025-01-28 00:00:00'

df['time'] = pd.to_datetime(df['time'])

mask = (df['coin'].isin(['HYPE/USDC', 'HYPE/USDH', 'HYPE/USDT'])) & (df['time'] > date) & (df['time'] < "2025-10-16 00:00:00")

subset = df.loc[mask].copy()

buys = subset[subset['dir'].str.lower() == 'buy']
sells = subset[subset['dir'].str.lower() == 'sell']

total_bought = buys['sz'].sum()
total_sold = sells['sz'].sum()

notional_bought = (buys['px'] * buys['sz']).sum()
avg_buy_price = notional_bought / total_bought if total_bought > 0 else 0

total_fee = abs(subset['fee'].sum())

net_hype = total_bought - total_sold

pnl = (current_price - avg_buy_price) * net_hype - total_fee

print(f"Total HYPE bought: {total_bought:.2f}")
print(f"Total HYPE sold: {total_sold:.2f}")
print(f"Net HYPE held: {net_hype:.2f}")
print(f"Average buy price: {avg_buy_price:.2f}")
print(f"Current price: {current_price:.2f}")
print(f"Unrealized PnL: ${pnl:.2f}")
print(f"Total fees: ${total_fee:.2f}")


opensea_pnl = 430.36
opensea_notional_pnl = opensea_pnl * current_price
print(f"------------------------------")
print(f"Opensea pnl: {opensea_pnl} HYPE")
print(f"Notional opensea pnl: ${opensea_notional_pnl:.2f}")
total_hype =  net_hype + opensea_pnl + net_loan_hype
total_pnl = pnl + loan_pnl
print(f"Total pnl: ${total_pnl:.2f}")
print(f"Net Hype Account: {total_hype:.2f} HYPE")

Total HYPE bought: 7076.57
Total HYPE sold: 4524.85
Net HYPE held: 2551.72
Average buy price: 44.29
Current price: 42.04
Unrealized PnL: $-5854.95
Total fees: $112.45
------------------------------
Opensea pnl: 430.36 HYPE
Notional opensea pnl: $18092.33
Total pnl: $-1749.73
Net Hype Account: 4770.85 HYPE
